In [1]:
!pip install fastembed

In [2]:
from fastembed import TextEmbedding

/home/jianqz666/miniconda3/envs/rag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Q1

In [37]:
embedder = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

In [38]:
query = "I just discovered the course. Can I join now?"

In [39]:
embedding = list(embedder.embed([query]))[0]

In [42]:
import numpy as np

In [44]:
embedding_np = np.array(embedding)
q1_result = np.min(embedding_np)
q1_result

np.float64(-0.11726373885183883)

## Q2

In [45]:
doc = 'Can I still join the course after the start date?'

In [46]:
embedding_2 = list(embedder.embed([doc]))[0]

In [49]:
embedding.dot(embedding_2)

np.float64(0.9008528895674548)

## Q3

In [50]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [63]:
texts = [doc['text'] for doc in documents]

In [64]:
texts

["Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
 "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'You can start by instal

In [66]:
text_embeddings = list(embedder.embed(texts))
query_embedding = list(embedder.embed([query]))[0]

In [68]:
V = np.array(text_embeddings)
Q = np.array(query_embedding)

In [71]:
cosine_similarities = V.dot(Q)

In [75]:
cosine_similarities

array([0.76296845, 0.81823782, 0.80853974, 0.71330788, 0.73044992])

In [74]:
print(np.argmax(cosine_similarities))

1


## Q4

In [76]:
texts_new = [doc['question'] + ' ' + doc['text'] for doc in documents]
texts_new

["Course - Can I still join the course after the start date? Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'Course - Can I follow the course after it finishes? Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
 "Course - When will the course start? The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin t

In [77]:
text_embeddings_new = list(embedder.embed(texts_new))

In [78]:
V_new = np.array(text_embeddings_new)
print(V_new.shape)

(5, 512)


In [79]:
cos_sims = V_new.dot(Q)
cos_sims

array([0.85145432, 0.8436594 , 0.84082872, 0.77551577, 0.80860079])

In [80]:
print(np.argmax(cos_sims))

0


## Q5

In [86]:
model_list = TextEmbedding.list_supported_models()
# model_list
dim_list = [model['dim'] for model in model_list]
print(min(dim_list))

384


## Q6

In [3]:
!python -m pip install -q "qdrant-client[fastembed]>=1.14.2"

In [4]:
from qdrant_client import QdrantClient, models

In [5]:
client = QdrantClient("http://localhost:6333") #connecting to local Qdrant instance

In [6]:
import requests

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

In [7]:
documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [8]:
EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [9]:
collection_name = "zoomcamp-hw2"

In [10]:
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

In [12]:
client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword"
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [13]:
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [15]:
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [90]:
query_points = client.query_points(
    collection_name=collection_name,
    query=models.Document(
        text=query,
        model=model_handle 
    ),
    limit=512,
    with_payload=True
)

# print(len(query_points.points), query_points.points[0])
scores = []

for point in query_points.points:
    scores.append(point.score)


In [91]:
print(max(scores))

0.8620738
